# CNN 😎

In [ ]:
import utilities as ff
import CNNModels

In [ ]:
# ff.delete_from_list()

In [ ]:
from  matplotlib import pyplot as plt
import numpy as np
from pathlib import Path, PurePath
import math


In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import *
from keras.models import *
from keras.losses import *
from keras.optimizers import *
from keras.utils import *


In [ ]:
from tensorflow import config as tfconfig
from tensorflow.python.client import device_lib
try:
    print(tfconfig.list_physical_devices())
    print(device_lib.list_local_devices())
except:
    pass


https://www.tensorflow.org/tutorials/images/cnn

## HyperParameter

Parameters that could in particualr affect performance of a CNN model:

- Image size
- Colors
- Number of filters
- Number of layers
- Kernel size of filters

We propose 2 models to evaluate the latter 3, but first of all:

### Base Model

In [ ]:
CNNModels.BASE().layers

### Thick model

- LESS LAYER
- MORE FILTERS
- ASCENDING KERNEL SIZE

In [ ]:
CNNModels.THICK().layers

### Long Model

- MORE LAYERS
- LESS FILTERS
- DESCENDING KERNEL SIZE

In [ ]:
CNNModels.LONG().layers

## Models

### Base

In [ ]:
base_tag, base_test = ff.auto_train("BASE")

### Grayscale

Are color useless?

In [ ]:
gray_tag, gray_test = ff.auto_train("BASE", color_mode="grayscale")

### Smaller images

Are high resolution images worth?

In [ ]:
small_tag, small_test = ff.auto_train("BASE", image_size=64)

### Thick

In [ ]:
thick_tag, thick_test = ff.auto_train("THICK")

### Long

In [ ]:
thick_tag, thick_test = ff.auto_train("LONG")

## Risk Estimation

Let's comprare best and worst model

In [ ]:
# BEST
best_accuracies = ff.k_fold_cross_validation("BEST", small_test, "LONG")
best_avg_loss = 1 - sum(best_accuracies) / len(best_accuracies)

print("Risk estimation (average zero one loss): ", best_avg_loss)

In [ ]:
# WORST
worst_accuracies = ff.k_fold_cross_validation("WORST", base_test, "THICK")
worst_avg_loss = 1 - sum(worst_accuracies) / len(worst_accuracies)

print("Risk estimation (average zero one loss): ", worst_avg_loss)